In [2]:
# Import Required Modules
from flask import Flask, render_template
import pandas as pd
import json
import plotly
import plotly.express as px
import pandas_ta as pta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import date
from datetime import datetime
from datetime import timedelta, date
from prophet import Prophet
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import mpld3
import matplotlib.pyplot as plt
import warnings

   
df = pd.read_csv("Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.Ticker('BTC-USD').history(period='7y',interval='1d',actions=False).reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
btc_df['date'] = btc_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
btc_df['date'] = pd.to_datetime(btc_df['date'])

df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Unnamed: 0', "wallets", "address", "mined"])
df['200D'] = df['price'].rolling(200).mean()
df['300D'] = df['price'].rolling(300).mean()
df['50D'] = df['price'].rolling(50).mean()
df['7D'] = df['price'].rolling(7).mean()
# df = df.dropna()
df['meanavge'] = (df['200D'] + df['300D'] + df['50D'] )/3
# df = df.drop(columns=['200D','300D', '50D'])
df['meanvalue'] = df["price"] - df["meanavge"]
df['status'] = df['meanvalue'].apply(lambda x: '1' if x > 0 else '0')
df['status']=df['status'].astype("object")
df['price-meanavge']=df['price'] - df['meanavge']
df['move%'] = (df['price-meanavge']/(df['price'] + df['meanavge']))
bins = [-.43, -.18, 0, .18, .43]
group_names = ["Severely Oversold","Neutral Oversold", "Neutral Overbought","Severely Overbought"]
df["Valuation"] = pd.cut(df["move%"], bins, labels=group_names)

c:\Users\chris\anaconda3\envs\python_data\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [3]:
df['daily_pct_change'] = df['price'].pct_change() * 100
df.tail(10)

,date,price,Open_x,High_x,Low_x,Volume_x,Open_y,High_y,Low_y,Volume_y,...,300D,50D,7D,meanavge,meanvalue,status,price-meanavge,move%,Valuation,daily_pct_change
4982,2024-06-13,66756.398438,68243.101562,68365.781250,66304.562500,2.895520e+10,68243.101562,68365.781250,66304.562500,2.895520e+10,...,47476.331406,65986.250625,68591.178571,56557.211380,10199.187057,1,10199.187057,0.082709,Neutral Overbought,-2.175796
4983,2024-06-14,66011.093750,66747.570312,67294.648438,65056.894531,2.740388e+10,66747.570312,67294.648438,65056.894531,2.740388e+10,...,47609.381035,66016.838359,68115.251116,56659.685378,9351.408372,1,9351.408372,0.076232,Neutral Overbought,-1.116454
4984,2024-06-15,66191.000000,66006.742188,66402.187500,65871.773438,1.412127e+10,66006.742188,66402.187500,65871.773438,1.412127e+10,...,47742.719089,66065.551953,67670.283482,56767.635783,9423.364217,1,9423.364217,0.076638,Neutral Overbought,0.272539
4985,2024-06-16,66639.046875,66189.359375,66894.843750,66018.250000,1.328114e+10,66189.359375,66894.843750,66018.250000,1.328114e+10,...,47877.768776,66129.950078,67240.434152,56882.085979,9756.960896,1,9756.960896,0.078990,Neutral Overbought,0.676900
4986,2024-06-17,66490.296875,66636.515625,67188.320312,65094.964844,3.000635e+10,66636.515625,67188.320312,65094.964844,3.000635e+10,...,48012.630911,66197.491406,66808.722098,56997.516385,9492.780490,1,9492.780490,0.076872,Neutral Overbought,-0.223217
4987,2024-06-18,65140.746094,66490.976562,66556.703125,64066.957031,3.948129e+10,66490.976562,66556.703125,64066.957031,3.948129e+10,...,48141.661263,66223.483906,66495.681362,57093.277329,8047.468765,1,8047.468765,0.065837,Neutral Overbought,-2.029696
4988,2024-06-19,64960.296875,65146.660156,65695.351562,64693.300781,2.110342e+10,65146.660156,65695.351562,64693.300781,2.110342e+10,...,48270.987676,66309.952734,66026.982701,57207.682350,7752.614525,1,7752.614525,0.063459,Neutral Overbought,-0.277014
4989,2024-06-20,64828.656250,64960.296875,66438.960938,64547.847656,2.564111e+10,64960.296875,66438.960938,64547.847656,2.564111e+10,...,48400.257637,66441.445625,65751.590960,57336.020410,7492.635840,1,7492.635840,0.061332,Neutral Overbought,-0.202648
4990,2024-06-21,64096.199219,64837.988281,65007.546875,63378.894531,2.618817e+10,64837.988281,65007.546875,63378.894531,2.618817e+10,...,48527.216758,66540.900938,65478.034598,57448.352057,6647.847161,1,6647.847161,0.054695,Neutral Overbought,-1.129835
4991,2024-06-22,64266.492188,64113.863281,64471.949219,63946.906250,1.871054e+10,64113.863281,64471.949219,63946.906250,1.871054e+10,...,48654.472754,66568.434063,65203.104911,57533.591504,6732.900684,1,6732.900684,0.055278,Neutral Overbought,0.265683


In [4]:
df = df[['date', 'price', 'daily_pct_change']]
df


,date,price,daily_pct_change
0,2010-09-25,0.100000,NaN
1,2010-09-26,0.100000,0.000000
2,2010-09-27,0.100000,0.000000
3,2010-09-28,0.100000,0.000000
4,2010-09-29,0.100000,0.000000
...,...,...,...
4987,2024-06-18,65140.746094,-2.029696
4988,2024-06-19,64960.296875,-0.277014
4989,2024-06-20,64828.656250,-0.202648
4990,2024-06-21,64096.199219,-1.129835


In [5]:
# Convert the dates to datetime objects
date1 = pd.to_datetime('2012-11-28')
date2 = pd.to_datetime('2016-07-9')
date3 = pd.to_datetime('2020-05-11')
date4 = pd.to_datetime('2024-04-20')

# Create the cycle DataFrames
cycle1 = df[df['date'] <= date1]
cycle2 = df[(df['date'] > date1) & (df['date'] <= date2)]
cycle3 = df[(df['date'] > date2) & (df['date'] <= date3)]
cycle4 = df[(df['date'] > date3) & (df['date'] <= date4)]
cycle5 = df[df['date'] > date4]

cycle1 = cycle1.reset_index(drop=True)
cycle2 = cycle2.reset_index(drop=True)
cycle3 = cycle3.reset_index(drop=True)
cycle4 = cycle4.reset_index(drop=True)
cycle5 = cycle5.reset_index(drop=True)


cycle1['cumulative_pct_change'] = cycle1['daily_pct_change'].cumsum()
cycle2['cumulative_pct_change'] = cycle2['daily_pct_change'].cumsum()
cycle3['cumulative_pct_change'] = cycle3['daily_pct_change'].cumsum()
cycle4['cumulative_pct_change'] = cycle4['daily_pct_change'].cumsum()
cycle5['cumulative_pct_change'] = cycle5['daily_pct_change'].cumsum()

cycle1 = cycle1.reset_index()
cycle2 = cycle2.reset_index()
cycle3 = cycle3.reset_index()
cycle4 = cycle4.reset_index()
cycle5 = cycle5.reset_index()

cycle1 = cycle1.rename(columns={'index': 'days from halving'})

cycle2 = cycle2.rename(columns={'index': 'days from halving'})
cycle3 = cycle3.rename(columns={'index': 'days from halving'})
cycle4 = cycle4.rename(columns={'index': 'days from halving'})
cycle5 = cycle5.rename(columns={'index': 'days from halving'})

cycle5



,days from halving,date,price,daily_pct_change,cumulative_pct_change
0,0,2024-04-21,64926.644531,-0.104312,-0.104312
1,1,2024-04-22,66837.679688,2.943376,2.839064
2,2,2024-04-23,66407.273438,-0.643957,2.195107
3,3,2024-04-24,64276.898438,-3.208045,-1.012938
4,4,2024-04-25,64481.707031,0.318635,-0.694303
...,...,...,...,...,...
58,58,2024-06-18,65140.746094,-2.029696,1.876033
59,59,2024-06-19,64960.296875,-0.277014,1.599018
60,60,2024-06-20,64828.656250,-0.202648,1.396370
61,61,2024-06-21,64096.199219,-1.129835,0.266535


In [6]:
# Get the first price value from cycle 5
first_price_cycle5 = cycle5['price'].iloc[0]

# cycle1['new_price'] = cycle1['price'].pct_change().add(1).fillna(1).cumprod().mul(first_price_cycle5)
cycles = [cycle1, cycle2, cycle3, cycle4]

for cycle in cycles:
    cycle['new_price'] = cycle['price'].pct_change().add(1).fillna(1).cumprod().mul(first_price_cycle5)
cycle1


,days from halving,date,price,daily_pct_change,cumulative_pct_change,new_price
0,0,2010-09-25,0.1,NaN,NaN,6.492664e+04
1,1,2010-09-26,0.1,0.000000,0.000000,6.492664e+04
2,2,2010-09-27,0.1,0.000000,0.000000,6.492664e+04
3,3,2010-09-28,0.1,0.000000,0.000000,6.492664e+04
4,4,2010-09-29,0.1,0.000000,0.000000,6.492664e+04
...,...,...,...,...,...,...
791,791,2012-11-24,12.4,0.000000,854.106991,8.050904e+06
792,792,2012-11-25,12.5,0.806452,854.913443,8.115831e+06
793,793,2012-11-26,12.3,-1.600000,853.313443,7.985977e+06
794,794,2012-11-27,12.2,-0.813008,852.500435,7.921051e+06


In [7]:
import pandas as pd
from datetime import timedelta

# Add new_date column to cycle3 DataFrame
cycle3['new_date'] = pd.to_datetime('2024-04-20') + pd.to_timedelta(cycle3['days from halving'], unit='D')
# cycle1['new_date'] = pd.to_datetime('2024-04-20') + pd.to_timedelta(cycle1['days from halving'], unit='D')
cycle2['new_date'] = pd.to_datetime('2024-04-20') + pd.to_timedelta(cycle2['days from halving'], unit='D')
# Add new_date column to cycle4 DataFrame
cycle4['new_date'] = pd.to_datetime('2024-04-20') + pd.to_timedelta(cycle4['days from halving'], unit='D')


In [8]:
import plotly.graph_objects as go
import pandas as pd

# Create a new figure
fig = go.Figure()

# Calculate the average of new_price for cycle3 and cycle4
average_price = pd.concat([cycle3['new_price'], cycle2['new_price'], cycle4['new_price']], axis=1).mean(axis=1)

# Convert the average_price Series into a DataFrame
average_price = average_price.to_frame()

# Rename the 0 column to 'new_price'
average_price.columns = ['new_price']

# Add a new column called 'new_date' and use the 'new_date' column from cycle4
average_price['new_date'] = cycle4['new_date']

# Add traces for new_price for cycle3 and cycle4
fig.add_trace(go.Scatter(x=cycle2.new_date, y=cycle2['new_price'], mode='lines', name='2012-2016 Cycle'))
fig.add_trace(go.Scatter(x=cycle3.new_date, y=cycle3['new_price'], mode='lines', name='2016-2020 Cycle'))
fig.add_trace(go.Scatter(x=cycle4.new_date, y=cycle4['new_price'], mode='lines', name='2020-2024 Cycle'))
fig.add_trace(go.Scatter(x=cycle5.date, y=cycle5['price'], mode='lines', name='2024-2028 Cycle', line=dict(color='black')))

# Set labels and title
fig.update_layout(
    title='Cycle Comparison Tracker',
    xaxis_title='Index',
    yaxis_title='New Price',
    width=1500,  # Set the width of the plot
    height=900  # Set the height of the plot
)

# Set y-axis to logarithmic scale and format ticks
fig.update_yaxes(type='log', tickformat='s')

# Show the plot
fig.show()

In [9]:
from datetime import date

# Get the most recent 'date' from cycle5
cycle5['date'] = pd.to_datetime(cycle5['date'])
recent_date = cycle5.iloc[-1]['date']

# Filter 'new_date' and 'new_price' for the most recent date
cycle2_today = cycle2[cycle2['new_date'] == pd.Timestamp(recent_date)][['new_date', 'new_price']]
cycle3_today = cycle3[cycle3['new_date'] == pd.Timestamp(recent_date)][['new_date', 'new_price']]
cycle4_today = cycle4[cycle4['new_date'] == pd.Timestamp(recent_date)][['new_date', 'new_price']]
cycle5_today = cycle5[cycle5['date'] == pd.Timestamp(recent_date)][['date', 'price']]

# Concatenate the dataframes and rename the 'new_price' or 'price' column
cycle2_today.rename(columns={'new_date': 'date', 'new_price': 'cycle2_price'}, inplace=True)
cycle3_today.rename(columns={'new_date': 'date', 'new_price': 'cycle3_price'}, inplace=True)
cycle4_today.rename(columns={'new_date': 'date', 'new_price': 'cycle4_price'}, inplace=True)
cycle5_today.rename(columns={'price': 'cycle5_price'}, inplace=True)

# Merge the dataframes on 'date'
compare_df = pd.merge(cycle2_today, cycle3_today, on='date', how='outer')
compare_df = pd.merge(compare_df, cycle4_today, on='date', how='outer')
compare_df = pd.merge(compare_df, cycle5_today, on='date', how='outer')

compare_df

,date,cycle2_price,cycle3_price,cycle4_price,cycle5_price
0,2024-06-22,106814.802293,60762.750623,68164.947299,64266.492188


In [10]:
import plotly.subplots as sp

# Create a 2x2 grid of subplots of type 'indicator'
fig = sp.make_subplots(rows=2, cols=2, subplot_titles=("2012 Scaled", "2016 Scaled", "2020 Scaled", "Current"), specs=[[{'type': 'indicator'}, {'type': 'indicator'}], [{'type': 'indicator'}, {'type': 'indicator'}]])

# Calculate the delta
delta = compare_df['cycle5_price'].iloc[-1]

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = compare_df['cycle2_price'].iloc[-1],
    delta = {'reference': delta}),
    row=1, col=1)

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = compare_df['cycle3_price'].iloc[-1],
    delta = {'reference': delta}),
    row=1, col=2)

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = compare_df['cycle4_price'].iloc[-1],
    delta = {'reference': delta}),
    row=2, col=1)

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = compare_df['cycle5_price'].iloc[-1],
    delta = {'reference': delta}),
    row=2, col=2)

fig.show()

In [11]:
days_df = df[['date', 'price']].copy()
import pandas as pd

# Define the bin ranges
bins = [0.01,0.1, 1, 10, 100, 1000, 10000, 100000]

# Create the bin labels
bin_labels = ['0.01-0.1','0.1-1', '1-10', '10-100', '100-1000', '1000-10000', '10000-100000']
# Cut the 'days_df' DataFrame by the bins and create a new column 'bin_label'
days_df['bin_label'] = pd.cut(days_df['price'], bins=bins, labels=bin_labels)

In [12]:
# Group by 'bin_labels' and count the occurrences
count_by_bin = days_df.groupby('bin_label').count()
count_by_bin = count_by_bin.rename(columns={"date": "Days within Price Range"})
count_by_bin = count_by_bin.drop(columns=["price"])
count_by_bin

C:\Users\chris\AppData\Local\Temp\ipykernel_11508\100618117.py:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,Days within Price Range
bin_label,
0.01-0.1,31
0.1-1,169
1-10,406
10-100,353
100-1000,1354
1000-10000,1095
10000-100000,1584


In [24]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Calculate the 200-week moving average
df['MA_200'] = df['price'].rolling(window=200).mean()
df['MA_50'] = df['price'].rolling(window=50).mean()

# Create two new dataframes based on the condition
df_above = df[df['MA_50'] > df['MA_200']]
df_below = df[df['MA_50'] <= df['MA_200']]

# Create a new figure
fig = go.Figure()

# Add the Close price trace with increased line width
fig.add_trace(go.Scatter(x=df['date'], y=df['price'], mode='lines', name='Price', line=dict(width=2)))

# # Add the 200-Day Moving Average trace with increased line width
# fig.add_trace(go.Scatter(x=df['date'], y=df['MA_200'], mode='lines', name='200-Day Moving Average', line=dict(color='red', width=4)))

# # Add the 50-Day Moving Average trace with increased line width
# fig.add_trace(go.Scatter(x=df['date'], y=df['MA_50'], mode='lines', name='50-Day Moving Average', line=dict(color='green', width=4)))

# Add the filled area between the 200-Day and 50-Day Moving Averages when MA_50 is above MA_200
fig.add_trace(go.Scatter(x=df_above['date'], y=df_above['MA_200'], mode='lines', line=dict(width=0), showlegend=False, fill=None))
fig.add_trace(go.Scatter(x=df_above['date'], y=df_above['MA_50'], mode='lines', fill='tonexty', line=dict(width=0), fillcolor='rgba(0,100,80,0.2)', showlegend=False))

# Add the filled area between the 200-Day and 50-Day Moving Averages when MA_50 is below MA_200
fig.add_trace(go.Scatter(x=df_below['date'], y=df_below['MA_200'], mode='lines', line=dict(width=0), showlegend=False, fill=None))
fig.add_trace(go.Scatter(x=df_below['date'], y=df_below['MA_50'], mode='lines', fill='tonexty', line=dict(width=0), fillcolor='rgba(255,0,0,0.2)', showlegend=False))

# Set the title and axis labels
fig.update_layout(
    title='BTC Close Price with 200, 50 -Day Moving Average',
    xaxis_title='Date',
    yaxis_title='Price',
    yaxis_type="log",
    autosize=False,
    width=1500,
    height=700,
)
fig.update_yaxes(type='log', tickformat='s')
# Show the plot
fig.show()